<a href="https://colab.research.google.com/github/Saaniya19/wildfirePredictorAI/blob/main/wildfirePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import os
import zipfile   # so we can unzip the files

local_zip = '/tmp/wildfire_data.zip'
# 'r' makes it a read-only file
zip_ref = zipfile.ZipFile(local_zip, 'r')
# unzipping the files
zip_ref.extractall('/tmp/')
zip_ref.close()

# defining the locations of the training and validation datasets
wildfire_train = os.path.join('/tmp/wildfire_data/train/wildfire')
nowildfire_train = os.path.join('/tmp/wildfire_data/train/nowildfire')
wildfire_validation = os.path.join('/tmp/wildfire_data/valid/wildfire')
nowildfire_validation = os.path.join('/tmp/wildfire_data/valid/nowildfire')

# printing number of images in each training and validation dataset
print('total wildfire training images:', len(os.listdir(wildfire_train)))
print('total non-wildfire training images:', len(os.listdir(nowildfire_train)))
print('total wildfire validation images:', len(os.listdir(wildfire_validation)))
print('total non-wildfire validation images:', len(os.listdir(nowildfire_validation)))

total wildfire training images: 7831
total non-wildfire training images: 7401
total wildfire validation images: 1741
total non-wildfire validation images: 1001


In [12]:
import tensorflow as tf
import keras.preprocessing
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# defining training data directory
TRAINING_DIR = "/tmp/wildfire_data/train/"
# generating more training data by zooming images
training_datagen = ImageDataGenerator(zoom_range=0.2)

# defining validation data directory
VALIDATION_DIR = "/tmp/wildfire_data/valid/"
# generating more validation data by zooming images
validation_datagen = ImageDataGenerator(zoom_range=0.2)

# defining training dataset
# image size is 100 x 100 pixels
# we can set class mode to categorical even though we only have two data classifications
# batch size is 500 meaning the number of training images utilized in a single iteration of the training algorithm
train_data = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 500
)

# defining validation dataset
valid_data = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 500,
)

Found 15230 images belonging to 2 classes.
Found 2740 images belonging to 2 classes.


In [14]:
from PIL import ImageFile

# the following line is so that we don't get an image truncated error
ImageFile.LOAD_TRUNCATED_IMAGES = True

model = tf.keras.models.Sequential([
    # the input shape is a 100 x 100 pixels image with 3 bytes color
    # this is the first convolution (with Max Pooling)
    # output shape is 2 because there are 2 classifications: wildfire and no wildfire
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # the second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # the third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # the fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten the results to feed into a DNN
    # dropout helps to improve the efficiency of the NN by throwing away unnescessary neurons
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # two dense layers in NN
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 2 because there are 2 classifications (wildfire and no wildfire)
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.summary()

model.compile(loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_data, epochs=5, steps_per_epoch=10, validation_data = valid_data, verbose = 1, validation_steps=1)

model.save("wildfire.h5")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 49, 49, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 47, 47, 64)        36928     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 10, 10, 128)     

In [40]:
import numpy as np
from google.colab import files
import keras.utils as image
import os
from os import listdir

pathFire = "/tmp/wildfire_data/test/wildfire/"
pathNoFire = "/tmp/wildfire_data/test/nowildfire/"
total = 0

img = image.load_img(pathFire+"-63.80422,51.85132.jpg", target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=1)
print(classes)

totalFire = 0
for i in os.listdir(pathFire):
  total += 1
  img = image.load_img(pathFire+i, target_size=(100, 100))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=1)
  print(classes)
  if classes[0][0]  < 0.95:
    totalFire += 1
  if total == 10:
    break

total = 0
totalNoFire = 0
for i in os.listdir(pathNoFire):
  total += 1
  img = image.load_img(pathNoFire+i, target_size=(100, 100))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=1)
  print(classes)
  if classes[0][0]  > 0.95:
    totalNoFire += 1
  if total == 10:
    break

print(str(totalFire) + "/10 photos of NO wildfires were predicted correctly")
print(str(totalNoFire) + "/10 photos of wildfires were predicted correctly")

1/1 [==============================] - 0s 32ms/step
[[0.720066   0.26950195]]
1/1 [==============================] - 0s 34ms/step
[[0.8001548  0.17070448]]
1/1 [==============================] - 0s 44ms/step
[[0.6738881  0.35178018]]
1/1 [==============================] - 0s 32ms/step
[[0.9307071  0.06519295]]
1/1 [==============================] - 0s 37ms/step
[[0.55712277 0.43958554]]
1/1 [==============================] - 0s 41ms/step
[[0.70348185 0.30295342]]
1/1 [==============================] - 0s 57ms/step
[[0.8799033  0.12098499]]
1/1 [==============================] - 0s 54ms/step
[[0.820153   0.17185427]]
1/1 [==============================] - 0s 44ms/step
[[0.85624146 0.134844  ]]
1/1 [==============================] - 0s 38ms/step
[[0.8412399  0.15961227]]
1/1 [==============================] - 0s 36ms/step
[[0.659602   0.32348153]]
1/1 [==============================] - 0s 36ms/step
[[0.9833637  0.01899157]]
1/1 [==============================] - 0s 62ms/step
[[0.9627257 